In [19]:
from langchain_groq import ChatGroq
import dotenv
import chromadb
import os
import pandas as pd

In [ ]:
llm = ChatGroq(temperature = 0,groq_api_key = "<apikey>",model = "llama-3.3-70b-versatile")

In [51]:
data = {
    "Project": [
        "SAR Image Colorization",
        "Real Time Fight Detection",
        "Explicit Content Detection",
        "Optical Content Detection",
        "Movie recommendation Systems",
        "Audio DeepFake Detection",
        "Fraud Detection Systems",
        "Question and Answering RAGs"
    ],
    "Technologies": [
        "GANs, PyTorch, Deep Learning, Satellite Images",
        "OpenCV, Deep Learning, LSTMs, Tensorflow, Scikit-learn",
        "Transformers, Attention Mechanisms, Deep Learning",
        "Computer Vision, PyTorch, Vision Models, OCR",
        "Scikit Learn, Recommendation Systems, Unsupervised Learning",
        "Deep Learning, PyTorch, Audio, LFCC, MFCC, Librosa",
        "Scikit Learn, Spacy, NLTK, HuggingFace, XGBoost",
        "LLMs, Llama, Generative AI, RAGs, Retrieval Augmented Generation"
    ],
    "Link": [
        "https://github.com/yaseeng-md/SAR_Image_Colorization",
        "https://github.com/yaseeng-md/Real_Time_Fight_Detection",
        "https://github.com/yaseeng-md/Explicit-Content-Detection",
        "https://github.com/yaseeng-md/FineTuning_TrOCR",
        "https://github.com/yaseeng-md/Movie_recommedation",
        "https://github.com/yaseeng-md/Audio-DeepFake-Detection",
        "https://github.com/yaseeng-md/Fraud-Detection-System",
        "https://github.com/yaseeng-md/Question-And-Answering-RAG"
    ]
}

df = pd.DataFrame(data)

In [47]:
from chromadb import PersistentClient
clinet = PersistentClient()
collection = clinet.create_collection(name = "yaseeng")

In [49]:
collection.add(
    documents=df["Technologies"].tolist(),
    metadatas=df[["Project", "Link"]].to_dict(orient="records"),
    ids=[str(i) for i in df.index]
)

In [55]:
from langchain_community.document_loaders import WebBaseLoader


In [102]:
def webLoader(path):
    loader = WebBaseLoader(web_path = path)
    docs = loader.load()
    return docs
docs = webLoader(path = "https://careers.nike.com/distribution-manager-early-late-shift/job/R-63911")

In [107]:
page_content = docs[0].page_content

In [167]:
import re

def clean_text(text):
    text = re.sub(r'<[^>]*?>', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = text.strip()
    text = ' '.join(text.split())
    return text

In [169]:
page_content_cleaned = clean_text(page_content)
print(page_content_cleaned)

Distribution Manager EarlyLate ShiftSkip to main contentOpen Virtual AssistantHomeCareer AreasTotal RewardsLifeNikePurposeLanguageSelect a Language Deutsch English Espaol Espaa Espaol Amrica Latina Franais Italiano Nederlands Polski Ting Vit Trke CareersClose MenuCareersChat Home Career Areas Total Rewards LifeNike Purpose Jordan CareersConverse CareersLanguageMenuReturn to Previous MenuSelect a Language Deutsch English Espaol Espaa Espaol Amrica Latina Franais Italiano Nederlands Polski Ting Vit Trke Back to Search Distribution Manager EarlyLate ShiftCategories IDCategories URLPosition TypeFull TimeDate PostedPrimary Quest IDSecond Quest IDJob ClassificationCareer areaDistribution Center OperationsLocationNieuwe Dijk 1 Ham Limburg 3945 BelgiumJob IDR63911 Apply Now Share JobShare Job PostingFacebookOpens In A New TabLinkedInLink Opens In New WindowEmailCloseMedium Default SizeiconClose MenuBecome a Part of the NIKE Inc TeamNIKE Inc does more than outfit the worlds best athletes It is 

In [171]:
from langchain_core.prompts import PromptTemplate
job_extraction_template = PromptTemplate.from_template(
            """
            ### SCRAPED TEXT FROM WEBSITE:
            {page_data}
            ### INSTRUCTION:
            The scraped text is from the career's page of a website.
            Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
            Only return the valid JSON.
            ### VALID JSON (NO PREAMBLE):
            """    
)


In [173]:
job_extract = job_extraction_template | llm

In [175]:
from langchain_core.output_parsers.json import JsonOutputParser


In [177]:
res = job_extract.invoke(input={"page_data": page_content_cleaned})

json_parser = JsonOutputParser()
res = json_parser.parse(res.content)


In [178]:
" ".join(res["skills"])

'TEAM MANAGEMENT ENGLISH Language skills Dutch language skills (in most instances) CHANGE AGILITY ANALYTICAL thinking NONHIERARCHICAL leadership'

In [230]:
res

{'role': 'Distribution Manager',
 'experience': 'Proven experience in leading teams, operations, logistics, transportation, retail/wholesale',
 'skills': ['TEAM MANAGEMENT',
  'ENGLISH Language skills',
  'Dutch language skills (in most instances)',
  'CHANGE AGILITY',
  'ANALYTICAL thinking',
  'NONHIERARCHICAL leadership'],
 'description': 'As a Distribution Manager, you will be working together with distribution manager colleagues on one of our Omnichannel Distribution Centers (DC) at our European Logistics Center (ELC) across Ham and Laakdal in Belgium. You will lead a team of Coaches (4 to 8 direct reports) who are our first line leaders of our athletes (employees) and drive our ELC purpose.'}

In [181]:
type(res)

dict

In [183]:
res_chroma = collection.query(query_texts=" ".join(res["skills"]),n_results=2)
res_chroma

{'ids': [['2', '7']],
 'embeddings': None,
 'documents': [['Transformers, Attention Mechanisms, Deep Learning',
   'LLMs, Llama, Generative AI, RAGs, Retrieval Augmented Generation']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'Project': 'Explicit Content Detection',
    'Link': 'https://github.com/yaseeng-md/Explicit-Content-Detection'},
   {'Project': 'Question and Answering RAGs',
    'Link': 'https://github.com/yaseeng-md/Question-And-Answering-RAG'}]],
 'distances': [[1.760313868522644, 1.8060634136199951]]}

In [219]:
projects = []
links = []
for repo in res_chroma["metadatas"][0]:
    projects.append(repo["Project"])
    links.append(repo["Link"])
    # print(repo["Project"])
projects,links

(['Explicit Content Detection', 'Question and Answering RAGs'],
 ['https://github.com/yaseeng-md/Explicit-Content-Detection',
  'https://github.com/yaseeng-md/Question-And-Answering-RAG'])

In [234]:
project_links = res_chroma["metadatas"][0]

In [238]:
mail_prompt = PromptTemplate.from_template(
            """
            ### JOB DESCRIPTION:
            {job_description}

            ### CANDIDATE PROFILE:
            You are Gandluru Mohammed Yaseen, an Arificial and Machine Learning Post Gradation Student, with experitice in Machine Learning and Generative AI.
            You are currently working upon LLM-based Article Summarization Tool using LLama 3B and you have a past experiece as Machine Learning Intern.
            You are a Research Student authored "Attension Based Explicit Content Detection - A Transformer based approcah".
            Your job is to write a cold email to the requirters regarding the job mentioned above describing the capability of you in fulfilling their needs.
            Also add the most relevant ones from the following links to showcase Yaseen's portfolio: {project_link}
            Remember you are Yaseen, Post Grad Student at LPU. 
            Do not provide a preamble.
            ### EMAIL (NO PREAMBLE):

            """
        )
mail_llm = mail_prompt | llm

In [242]:
res_email = mail_llm.invoke(input={"job_description" : res, "project_link" : project_links })

In [250]:
print(res_email.content)

Subject: Application for Distribution Manager Role at European Logistics Center

Dear Hiring Manager,

I am excited to apply for the Distribution Manager position at your esteemed organization, as advertised. Although my background is in Artificial Intelligence and Machine Learning, I am confident that my skills in team management, analytical thinking, and adaptability make me an ideal candidate for this role.

As a Post Gradation Student in Artificial and Machine Learning at Lovely Professional University, I have developed strong analytical and problem-solving skills, which I believe are essential for a Distribution Manager. My experience as a Machine Learning Intern has taught me the importance of teamwork, leadership, and effective communication. I am eager to leverage these skills in a new challenge and contribute to the success of your team.

I am particularly drawn to this role because of the emphasis on team management, change agility, and non-hierarchical leadership. My researc